In [2]:
import ast
import imdb
import json
import pickle
import itertools
import dateparser
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ast import literal_eval
from price_parser import Price
from collections import Counter

%matplotlib inline

ModuleNotFoundError: No module named 'imdb'

In [ ]:
FOLDER_PATH = 'data/'

## Load Data ##

In [ ]:
# Data fetched from the IMDB API and stored on our SQL database
DATA_COLUMNS = ["index","wikipedia_id", "imdb_id", "title", "cast", "genres", "runtimes", "countries", "languages", "box_office", "rating", "votes", "plot", "synopsis"]
df = pd.read_csv("./Data/fetched_data.csv", names=DATA_COLUMNS, header=0, index_col=0)

In [ ]:
# formate data fetched from our database 
#format the title
# TODO (Martin) => clean a bit the following lines
df["title"] = df["title"].astype("string")
df.cast = df.cast.apply(lambda x: literal_eval(x))
df.genres = df.genres.apply(lambda x: [] if pd.isna(x) else literal_eval(x))
df["runtimes"] = pd.to_numeric(df["runtimes"].apply(lambda x: "0" if pd.isna(x) else x.replace("[", "").replace("]", "").replace("'", "")))
df.countries = df.countries.apply(lambda x: [] if pd.isna(x) else literal_eval(x))
df.languages = df.languages.apply(lambda x: [] if pd.isna(x) else literal_eval(x))
df["synopsis"] = df["synopsis"].apply(lambda x: "" if pd.isna(x) else literal_eval(x)[0])
df["plot"] = df["plot"].apply(lambda x: "" if pd.isna(x) else literal_eval(x)[0])

In [ ]:
# TODO question assistant => comment avoir la bonne valeur de box office en dollar 
# sachant que le cours a evolue au cours du temps

def convert_currency(x):
    formated = Price.fromstring(x)
    amount = formated.amount_float
    match formated.currency:
        case "$":
            return amount
        case "EUR":
            return amount * 1.18
        case "GBP":
            return amount * 1.36
        case "SEK":
            return amount * 0.11
        case "CAD":
            return amount * 0.79
        case "INR":
            return amount * 0.013
        case "CZK":
            return amount * 0.04
        case "JPY":
            return amount * 0.009
        case _:
            return None
    

In [ ]:
# Format the box office column
def format_box_office(box_office):
    if pd.isna(box_office):
        return None, None, None

    film = ast.literal_eval(box_office)
    budget = convert_currency(film["Budget"]) if "Budget" in film else None

    bo_usa = Price.fromstring(film["Opening Weekend United States"]).amount if "Opening Weekend United States" in film else None
    bo_world = Price.fromstring(film["Cumulative Worldwide Gross"]).amount if "Cumulative Worldwide Gross" in film else None

    return budget, bo_usa, bo_world
    
formated_bo = df["box_office"].apply(lambda x: format_box_office(x))
df["budget"], df["box_office_usa"], df["box_office_world"] = zip(*formated_bo)
df.drop(columns=["box_office"], inplace=True)

#### Load Character Metadata ####

In [ ]:
COLUMNS_NAMES_CHARACTERS = [
    "wiki_movie_id",
    "freebase_movie_id",
    "movie_release_date",
    "character_name",
    "actor_date_of_birth",
    "actor_gender",
    "actor_height",
    "actor_ethnicity",
    "actor_name",
    "actor_age_at_movie_release",
    "freebase_character_actor_map_id",
    "freebase_character_id",
    "freebase_actor_id"
]
df_character = pd.read_csv(FOLDER_PATH + 'character.metadata.tsv', sep='\t', header=None, names=COLUMNS_NAMES_CHARACTERS)
df_character.head()

,wiki_movie_id,freebase_movie_id,movie_release_date,character_name,actor_date_of_birth,actor_gender,actor_height,actor_ethnicity,actor_name,actor_age_at_movie_release,freebase_character_actor_map_id,freebase_character_id,freebase_actor_id
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg


#### Load Movie Metadata ####

In [ ]:
COLUMNS_NAMES_MOVIES = [
    "wiki_movie_id",
    "freebase_movie_id",
    "movie_name",
    "movie_release_date",
    "movie_box_office_revenue",
    "movie_runtime",
    "movie_languages",
    "movie_countries",
    "movie_genres"
]
df_movie = pd.read_csv(FOLDER_PATH + 'movie.metadata.tsv', sep='\t', header=None, names=COLUMNS_NAMES_MOVIES)

In [ ]:
def extract_languages(array_of_languages):
    return [x.split(' ')[0] for x in array_of_languages]

# Treats the languages object to turn it into a list of languages
df_movie["languages"] = df_movie.movie_languages.apply(lambda x: json.loads(x).values()).apply(lambda x: list(x)).apply(lambda x: [] if x==[] else extract_languages(x))
df_movie.drop("movie_languages", axis=1, inplace=True)

# Treats the countries object to turn it into a list of countries
df_movie["countries"] = df_movie.movie_countries.apply(lambda x: json.loads(x).values()).apply(lambda x: list(x)).apply(lambda x: [] if x==[] else x)
df_movie.drop("movie_countries", axis=1, inplace=True)

# Treats the genres object to turn it into a list of genres
df_movie["genres"] = df_movie.movie_genres.apply(lambda x: json.loads(x).values()).apply(lambda x: list(x)).apply(lambda x: [] if x==[] else x)
df_movie.drop("movie_genres", axis=1, inplace=True)

In [ ]:
df_movie.head(5)

,wiki_movie_id,freebase_movie_id,movie_name,movie_release_date,movie_box_office_revenue,movie_runtime,languages,countries,genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,[English],[United States of America],"[Thriller, Science Fiction, Horror, Adventure,..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,[English],[United States of America],"[Mystery, Biographical film, Drama, Crime Drama]"
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,[Norwegian],[Norway],"[Crime Fiction, Drama]"
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,[English],[United Kingdom],"[Thriller, Erotic thriller, Psychological thri..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,[German],[Germany],[Drama]


#### Load the characters clusters ####

In [ ]:
COLUMNS_NAMES_NAMES = [
    "character_name",
    "freebase_character_actor_map_id"
]
df_name = pd.read_csv(FOLDER_PATH + 'name.clusters.txt', sep='\t', header=None, names=COLUMNS_NAMES_NAMES)
df_name.head()

,character_name,freebase_character_actor_map_id
0,Stuart Little,/m/0k3w9c
1,Stuart Little,/m/0k3wcx
2,Stuart Little,/m/0k3wbn
3,John Doe,/m/0jyg35
4,John Doe,/m/0k2_zn


#### Load the plots ####

In [ ]:
COLUMNS_NAMES_PLOTS = [
    "wiki_movie_id",
    "plot"
]
df_plot = pd.read_csv(FOLDER_PATH + 'plot_summaries.txt', sep='\t', header=None, names=COLUMNS_NAMES_PLOTS, index_col=0)
df_plot.head()

,plot
wiki_movie_id,
23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
31186339,The nation of Panem consists of a wealthy Capi...
20663735,Poovalli Induchoodan is sentenced for six yea...
2231378,"The Lemon Drop Kid , a New York City swindler,..."
595909,Seventh-day Adventist Church pastor Michael Ch...


#### Load the characters types ####

In [ ]:
COLOMNS_NAMES_TV_TROPES = [
    "character_type",
    "object"
]
df_tvtropes = pd.read_csv(FOLDER_PATH + 'tvtropes.clusters.txt', sep='\t', header=None, names=COLOMNS_NAMES_TV_TROPES)

In [ ]:
# Treats the object column to turn it into multiple columns
df_tvtropes["character_name"] = df_tvtropes.apply(lambda x: json.loads(x["object"])["char"], axis=1)
df_tvtropes["movie_name"] = df_tvtropes.apply(lambda x: json.loads(x["object"])["movie"], axis=1)
df_tvtropes["freebase_id"] = df_tvtropes.apply(lambda x: json.loads(x["object"])["id"], axis=1)
df_tvtropes["actor_name"] = df_tvtropes.apply(lambda x: json.loads(x["object"])["actor"], axis=1)
df_tvtropes.drop("object", axis=1, inplace=True)

In [ ]:
df_tvtropes.head()

,character_type,character_name,movie_name,freebase_id,actor_name
0,absent_minded_professor,Professor Philip Brainard,Flubber,/m/0jy9q0,Robin Williams
1,absent_minded_professor,Professor Keenbean,Richie Rich,/m/02vchl3,Michael McShane
2,absent_minded_professor,Dr. Reinhardt Lane,The Shadow,/m/0k6fkc,Ian McKellen
3,absent_minded_professor,Dr. Harold Medford,Them!,/m/0k6_br,Edmund Gwenn
4,absent_minded_professor,Daniel Jackson,Stargate,/m/0k3rhh,James Spader


#### Combine dataframes ####

In [ ]:
# df is the dataframe containing what we fetched from the API
df.head()

,wikipedia_id,imdb_id,title,cast,genres,runtimes,countries,languages,rating,votes,plot,synopsis,budget,box_office_usa,box_office_world
index,,,,,,,,,,,,,,,
0,975900,228333,ghosts of mars,"[Natasha Henstridge, Ice Cube, Jason Statham, ...","[Action, Horror, Sci-Fi]",98,[United States],[English],4.9,55370,"In 2176, a Martian police unit is sent to pick...","200 years in the future, Mars has been coloniz...",28000000.0,None,None
1,28463795,94806,brun bitter,"[Frank Krog, Kristin Kajander, Anne Krigsvoll,...","[Crime, Drama]",83,[Norway],[Norwegian],5.7,40,"The lonely, divorced and disillusioned lawyer ...",,NaN,None,None
2,261236,83949,a woman in flames,"[Gudrun Landgrebe, Mathieu Carrière, Hanns Zis...",[Drama],106,[West Germany],[German],6.0,591,"Eva, an upper-class housewife, frustratingly l...",,NaN,None,None
3,10408933,29852,alexander's ragtime band,"[Tyrone Power, Alice Faye, Don Ameche, Ethel M...","[Drama, Music, Musical, Romance]",106,[United States],[English],6.9,2164,This send-up of ragtime song and dance begins ...,,2000000.0,None,None
4,175026,21335,sarah and son,"[Ruth Chatterton, Fredric March, Fuller Mellis...","[Drama, Romance]",86,[United States],[English],5.4,274,"After years of abusing his wife, a ne'er-do-we...",,NaN,None,None


In [ ]:
interesting_cols_scraped = df[["wikipedia_id", "imdb_id", "cast", "rating", "votes", "budget"]]
interesting_cols_scraped.rename(columns={"wikipedia_id": "wiki_movie_id"}, inplace=True)

/var/folders/07/9l9kqd013xjdlz8_0b2__8xr0000gq/T/ipykernel_41030/985237478.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interesting_cols_scraped.rename(columns={"wikipedia_id": "wiki_movie_id"}, inplace=True)


In [ ]:
interesting_cols_scraped.sample(5)

,wiki_movie_id,imdb_id,cast,rating,votes,budget
index,,,,,,
7974,5622643,106986,"[Jackie Shroff, Amrish Puri, Dimple Kapadia, F...",7.6,890,NaN
26139,13632871,341855,"[Gérard Depardieu, Michael York, Nastassja Kin...",5.5,1097,NaN
19540,11156173,26121,"[Paul Muni, Karen Morley, William Gargan, Bart...",6.4,979,NaN
34551,24268260,54391,"[Tommy Steele, Janet Munro, Sidney James, Bern...",5.0,145,NaN
2355,573304,107156,"[Ah-Lei Gua, Sihung Lung, May Chin, Winston Ch...",7.6,15901,1000000.0


In [ ]:
df_movie_final = pd.merge(df_movie, interesting_cols_scraped, on="wiki_movie_id", how="left").merge(df_plot, on="wiki_movie_id", how="inner")

In [ ]:
# Add characters names list fron the character db
# We want to have a list of characters names for each movie
# TODO => fix bug with NAN values appearing in the list
df_movie_final["characters_names"] = df_movie_final.wiki_movie_id.apply(lambda x: df_character[df_character.wiki_movie_id == x].character_name.tolist())

#### Add main character for each movie ####

In [ ]:
df_movie_final.characters_names

0        [Akooshay, Lieutenant Melanie Ballard, Desolat...
1                                               [nan, nan]
2                                          [nan, nan, nan]
3             [nan, Morgana, nan, nan, nan, nan, nan, nan]
4                      [nan, Rebecca, Nina, nan, nan, nan]
                               ...                        
42199    [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
42200                                                   []
42201                                                   []
42202    [Mesena, Hibiki, Ishtar, Sylvie, Exegran, Addi...
42203             [nan, nan, nan, nan, nan, nan, nan, nan]
Name: characters_names, Length: 42204, dtype: object

In [ ]:
# Find main character
# TODO => fix it: not working super well yet because of bugs in the character names list
# Stop_words = ["Jr", "J.", "W.", "Dr."] # TODO => add more stop words with regex to remove types *. 
def find_main_character(characters, plot):
    if characters == []:
        return None
    if pd.isna(plot):
        return None
    occurences = []

    for character in characters:
        if not pd.isna(character):
            possible_names = character.split()
            local_count = 0
            for name in possible_names:
                local_count += plot.count(name)
            occurences.append(local_count)
    if (occurences) == []:
        return None

    return characters[occurences.index(max(occurences))]

In [ ]:
df_movie_final["main_character"] = df_movie_final.apply(lambda x: find_main_character(x.characters_names, x["plot"]), axis=1)

#### Cleaning release dates

In [ ]:
# Finally treat release date to have only the year (so it is uniform over the db)
# For each movies we have in df, find by id the one in years df
def parse_date(x):
    if pd.isna(x):
        return None
    year = dateparser.parse(x).year
    if year < 1893:
        return None
    return year

df_movie_final["release_year"] = df_movie_final.movie_release_date.apply(lambda x: parse_date(x))

In [ ]:
df_movie_final.drop("movie_release_date", axis=1, inplace=True)

#### Saving clean dataset

In [ ]:
df_movie_final.to_csv(FOLDER_PATH + "movie_processed.csv")
df_tvtropes.to_csv(FOLDER_PATH + "tvtropes_processed.csv")

In [ ]:
df_movie_final.columns

Index(['wiki_movie_id', 'freebase_movie_id', 'movie_name',
       'movie_box_office_revenue', 'movie_runtime', 'languages', 'countries',
       'genres', 'imdb_id', 'cast', 'rating', 'votes', 'budget', 'plot',
       'characters_names', 'main_character', 'release_year'],
      dtype='object')